In [1]:
import xarray as xr 

# ds = xr.open_dataset('/teamspace/studios/this_studio/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/f0e93607df96295f1c9e14ccfe96032f.nc')
ds = xr.open_dataset('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/f0e93607df96295f1c9e14ccfe96032f.nc')

In [2]:
ds

<xarray.Dataset> Size: 11GB
Dimensions:    (date: 405, latitude: 241, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 3kB 19910101 19910201 19910301 ... 20240801 20240901
  * latitude   (latitude) float64 2kB 30.0 29.75 29.5 ... -29.5 -29.75 -30.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    expver     (date) <U4 6kB ...
Data variables: (12/20)
    t2m        (date, latitude, longitude) float32 562MB ...
    d2m        (date, latitude, longitude) float32 562MB ...
    msl        (date, latitude, longitude) float32 562MB ...
    sp         (date, latitude, longitude) float32 562MB ...
    sst        (date, latitude, longitude) float32 562MB ...
    skt        (date, latitude, longitude) float32 562MB ...
    ...         ...
    lai_hv     (date, latitude, longitude) float32 562MB ...
    cvl        (date, latitude, longitude) float32 562MB ...
    tcc        (date, latitude, longitude) float32 562MB ...
    mper       (date, latitude, longitude) float32 562MB ...
    tco3       (date, latitude, longitude) float32 562MB ...
    lsm        (date, latitude, longitude) float32 562MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-17T18:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-30, 30.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(-180, 180, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if ds.latitude[0] > ds.latitude[-1]:
    ds = ds.sortby("latitude")

# Step 4: Perform interpolation
ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")


In [4]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 405, latitude: 61, longitude: 360)
Coordinates:
    number     int64 8B 0
  * date       (date) int64 3kB 19910101 19910201 19910301 ... 20240801 20240901
    expver     (date) <U4 6kB '0001' '0001' '0001' ... '0001' '0005' '0005'
  * latitude   (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude  (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables: (12/20)
    t2m        (date, latitude, longitude) float64 71MB 295.4 295.2 ... 299.1
    d2m        (date, latitude, longitude) float64 71MB 289.9 289.7 ... 295.5
    msl        (date, latitude, longitude) float64 71MB 1.016e+05 ... 1.019e+05
    sp         (date, latitude, longitude) float64 71MB 1.016e+05 ... 1.019e+05
    sst        (date, latitude, longitude) float64 71MB 297.4 296.8 ... 300.2
    skt        (date, latitude, longitude) float64 71MB 297.1 296.6 ... 300.0
    ...         ...
    lai_hv     (date, latitude, longitude) float64 71MB 0.0 0.0 0.0 ... 0.0 0.0
    cvl        (date, latitude, longitude) float64 71MB 0.0 0.0 0.0 ... 0.0 0.0
    tcc        (date, latitude, longitude) float64 71MB 0.5719 0.54 ... 0.373
    mper       (date, latitude, longitude) float64 71MB 2.692e-09 ... 9.604e-10
    tco3       (date, latitude, longitude) float64 71MB 0.005744 ... 0.006194
    lsm        (date, latitude, longitude) float64 71MB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-17T18:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
import pandas as pd

# 1. Convert 'date' coordinate to datetime
ds['date'] = pd.to_datetime(ds['date'].astype(str), format='%Y%m%d')

In [6]:
# import zipfile
# import os

# # Specify the path to the zip file
# zip_file_path = 'ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/6268e3986ce330c33332a86dd878fba4.zip'

# # Define the directory where you want to extract the files
# extract_to_path = 'ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/extracted_files/'

# # Create the directory if it doesn't exist
# os.makedirs(extract_to_path, exist_ok=True)

# # Open the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     # Extract all the contents into the specified directory
#     zip_ref.extractall(extract_to_path)

# print(f'Files extracted to: {extract_to_path}')


In [7]:
import xarray as xr
import os

# Path to the directory containing the NetCDF files
data_dir = '/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/extracted_files/'

# Search for all .nc files in the directory
nc_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.nc')]

In [8]:
# Use xarray to open multiple NetCDF files
sm = xr.open_mfdataset(nc_files, combine='by_coords')

In [9]:
sm

<xarray.Dataset> Size: 8GB
Dimensions:     (lat: 720, lon: 1440, time: 376)
Coordinates:
  * lat         (lat) float32 3kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time        (time) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
Data variables:
    sm          (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sensor      (time, lat, lon) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    freqbandID  (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    nobs        (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [10]:
sm = sm.rename({'lat': 'latitude', 'lon': 'longitude', 'time': 'date'})

In [11]:
import xarray as xr
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from typing import Optional, List
from datetime import datetime

def visualize_variable_on_map(
    dataset: xr.Dataset,
    variable: str,
    time_dim: str = 'date',
    lat_dim: str = 'latitude',
    lon_dim: str = 'longitude',
    plot_type: str = 'scatter_geo',  # Options: 'scatter_geo', 'imshow'
    downsample_factor: Optional[int] = 1,
    projection: str = 'natural earth',
    color_scale: str = 'Viridis',
    title: Optional[str] = None,
    animation_frame: Optional[str] = None,
    hover_precision: int = 2,
    custom_colorbar_title: Optional[str] = None,
    start_date: Optional[str] = None,  # New parameter for start date
    end_date: Optional[str] = None,    # New parameter for end date
    **kwargs
) -> go.Figure:
    """
    Universal function to visualize any variable from an xarray Dataset on a map with optional animation using Plotly.
    
    Parameters:
    - dataset (xr.Dataset): The xarray Dataset containing the data.
    - variable (str): The name of the variable to visualize.
    - time_dim (str): Name of the time dimension. Default is 'time'.
    - lat_dim (str): Name of the latitude dimension. Default is 'lat'.
    - lon_dim (str): Name of the longitude dimension. Default is 'lon'.
    - plot_type (str): Type of plot. Options include 'scatter_geo' and 'imshow'. Default is 'scatter_geo'.
    - downsample_factor (int, optional): Factor by which to downsample spatial data for performance. Default is 1 (no downsampling).
    - projection (str): Map projection style for Plotly. Default is 'natural earth'.
    - color_scale (str or list): Color scale for the plot. Default is 'Viridis'.
    - title (str, optional): Title of the plot. If None, a default title is generated.
    - animation_frame (str, optional): Dimension to animate over. If None, no animation is created. Default is None.
    - hover_precision (int): Decimal precision for hover data. Default is 2.
    - custom_colorbar_title (str, optional): Custom title for the color bar. If None, the variable name is used.
    - start_date (str, optional): Start date for the data in ISO format (YYYY-MM-DD). If None, no start date filtering is applied.
    - end_date (str, optional): End date for the data in ISO format (YYYY-MM-DD). If None, no end date filtering is applied.
    - **kwargs: Additional keyword arguments passed to the Plotly plotting functions.
    
    Returns:
    - fig (plotly.graph_objs._figure.Figure): The Plotly figure object.
    """
    
    # Validate plot_type
    valid_plot_types = ['scatter_geo', 'imshow']
    if plot_type not in valid_plot_types:
        raise ValueError(f"Invalid plot_type '{plot_type}'. Valid options are: {valid_plot_types}")
    
    # Check if the variable exists
    if variable not in dataset.data_vars:
        raise ValueError(f"Variable '{variable}' not found in the dataset.")
    
    # Extract the DataArray
    da = dataset[variable]
    
    # Ensure required dimensions are present
    for dim in [time_dim, lat_dim, lon_dim]:
        if dim not in da.dims:
            raise ValueError(f"Dimension '{dim}' not found in variable '{variable}'.")
    
    # Convert time dimension to datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(da[time_dim].dtype):
        da[time_dim] = pd.to_datetime(da[time_dim].values)
    
    # Apply date range filtering if specified
    if start_date or end_date:
        if start_date:
            try:
                start_datetime = pd.to_datetime(start_date)
                da = da.sel({time_dim: da[time_dim] >= start_datetime})
            except Exception as e:
                raise ValueError(f"Invalid start_date '{start_date}': {e}")
        if end_date:
            try:
                end_datetime = pd.to_datetime(end_date)
                da = da.sel({time_dim: da[time_dim] <= end_datetime})
            except Exception as e:
                raise ValueError(f"Invalid end_date '{end_date}': {e}")
    
    # Check if the dataset is empty after filtering
    if da.date.size == 0:
        raise ValueError("No data available for the specified date range.")
    
    # Downsample if required
    if downsample_factor > 1:
        da = da.coarsen({lat_dim: downsample_factor, lon_dim: downsample_factor}, boundary='trim').mean()
    
    # Handle animation
    animate = False
    if animation_frame and animation_frame in da.dims:
        animate = True
    elif animation_frame:
        raise ValueError(f"Animation frame dimension '{animation_frame}' not found in variable '{variable}'.")
    
    # Generate title if not provided
    if not title:
        if start_date and end_date:
            title = f"{variable} from {start_date} to {end_date}"
        elif start_date:
            title = f"{variable} from {start_date} onwarsm"
        elif end_date:
            title = f"{variable} up to {end_date}"
        else:
            title = f"{variable} Visualization"
    
    if plot_type == 'scatter_geo':
        # Convert to DataFrame
        df = da.to_dataframe().reset_index()
    
        # Drop NaNs
        df = df.dropna(subset=[variable])
    
        # Convert animation frame to string if animating
        if animate:
            df[animation_frame] = df[animation_frame].astype(str)
        else:
            # If not animating, set a default frame
            df['Frame'] = 'Frame'
    
        # Create the scatter_geo plot
        fig = px.scatter_geo(
            df,
            lat=lat_dim,
            lon=lon_dim,
            color=variable,
            animation_frame=animation_frame if animate else 'Frame',
            projection=projection,
            color_continuous_scale=color_scale,
            title=title,
            labels={variable: variable.upper()},
            hover_data={variable: f':.{hover_precision}f'},
            **kwargs
        )
    
        # Update layout for better appearance
        fig.update_layout(
            coloraxis_colorbar=dict(
                title=custom_colorbar_title if custom_colorbar_title else variable.upper(),
                ticks="outside"
            )
        )
    
        # If not animating, remove animation controls
        if not animate:
            fig.update_layout(
                updatemenus=[]
            )
    
    elif plot_type == 'imshow':
        # For imshow, we need to handle each frame separately
        if animate:
            # Prepare frames
            frames = []
            times = da[animation_frame].values
            for t in times:
                da_t = da.sel({animation_frame: t})
                # Convert to numpy array
                z = da_t.values
                # Handle latitude descending for proper display
                if da[lat_dim][0] > da[lat_dim][-1]:
                    z = z[::-1, :]
                frames.append(go.Frame(
                    data=[go.Heatmap(
                        z=z,
                        x=da[lon_dim].values,
                        y=da[lat_dim].values,
                        colorscale=color_scale,
                        **kwargs
                    )],
                    name=str(t)
                ))
    
            # Initial frame
            initial_t = times[0]
            initial_da = da.sel({animation_frame: initial_t})
            z_init = initial_da.values
            if da[lat_dim][0] > da[lat_dim][-1]:
                z_init = z_init[::-1, :]
    
            # Create initial heatmap
            fig = go.Figure(
                data=go.Heatmap(
                    z=z_init,
                    x=da[lon_dim].values,
                    y=da[lat_dim].values,
                    colorscale=color_scale,
                    **kwargs
                ),
                layout=go.Layout(
                    title=title,
                    updatemenus=[dict(
                        type='buttons',
                        buttons=[
                            dict(label='Play',
                                 method='animate',
                                 args=[None, {"frame": {"duration": 500, "redraw": True},
                                              "fromcurrent": True}]),
                            dict(label='Pause',
                                 method='animate',
                                 args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                "mode": "immediate",
                                                "transition": {"duration": 0}}])
                        ],
                        showactive=False,
                        x=0.1,
                        y=0
                    )]
                ),
                frames=frames
            )
    
            # Update color bar
            fig.update_traces(colorbar=dict(
                title=custom_colorbar_title if custom_colorbar_title else variable.upper(),
                ticks="outside"
            ))
    
        else:
            # Single heatmap without animation
            da_t = da
            z = da_t.values
            if da[lat_dim][0] > da[lat_dim][-1]:
                z = z[::-1, :]
    
            fig = px.imshow(
                z,
                labels=dict(x=lon_dim, y=lat_dim, color=variable),
                x=da[lon_dim].values,
                y=da[lat_dim].values,
                color_continuous_scale=color_scale,
                title=title,
                **kwargs
            )
    
            # Update color bar
            fig.update_layout(
                coloraxis_colorbar=dict(
                    title=custom_colorbar_title if custom_colorbar_title else variable.upper(),
                    ticks="outside"
                )
            )
    
    else:
        raise NotImplementedError(f"Plot type '{plot_type}' is not implemented.")
    
    return fig


In [12]:
# Example Usage:
# Assuming you have your xarray Dataset loaded as `sm`

# Visualize 'sm' with animation using scatter_geo and a specified date range
fig = visualize_variable_on_map(
    dataset=sm,
    variable='sm',
    plot_type='scatter_geo',
    animation_frame='date',
    downsample_factor=5,
    title='Soil Moisture Over Time',
    start_date='2020-01-01',  # Specify the start date
    end_date='2020-01-31',    # Specify the end date
    color_scale = [
        [0.0, "darkred"],        # 0% - extremely dry
        [0.1, "red"],            # 10% - very dry
        [0.2, "orangered"],      # 20% - dry
        [0.3, "lightgreen"],     # 30% - beginning of optimal moisture
        [0.4, "limegreen"],      # 40% - optimal moisture for most crops
        [0.5, "green"],          # 50% - middle of optimal moisture
        [0.55, "darkseagreen"],  # 55% - upper limit of optimal moisture
        [0.6, "darkgreen"],      # 60% - end of optimal moisture
        [0.7, "lightblue"],      # 70% - moist but not excessive
        [0.8, "skyblue"],        # 80% - very moist
        [0.9, "deepskyblue"],    # 90% - extremely moist
        [1.0, "blue"]            # 100% - very wet
    ]
)
fig.show()

In [13]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-30, 30.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(-180, 180, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if sm.latitude[0] > sm.latitude[-1]:
    sm = sm.sortby("latitude")

# Step 4: Perform interpolation
sm = sm.interp(latitude=target_lat, longitude=target_lon, method="linear")

In [14]:
# sm = sm.fillna(2)

In [15]:
sm = sm.load()

In [16]:
sm

<xarray.Dataset> Size: 264MB
Dimensions:     (date: 376, latitude: 61, longitude: 360)
Coordinates:
  * date        (date) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
  * latitude    (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude   (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables:
    sm          (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    sensor      (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    freqbandID  (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    nobs        (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [17]:
ds_merged = xr.merge([ds, sm])

In [18]:
# ds_merged = ds_merged.fillna(0)

In [19]:
# Convert the dates to strings to filter
start_date = '2000-01-01 '
end_date = '2022-12-01'

# Select the date range in the dataset
ds = ds_merged.sel(date=slice(start_date, end_date))

In [20]:
ds = ds.drop_vars(['sensor', 'freqbandID', 'nobs'])

In [22]:
ds.to_netcdf('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/02_intermediate/merged_data.nc')